### Polygonal ROI

* This code has been taken from: https://stackoverflow.com/questions/37099262/drawing-filled-polygon-using-mouse-events-in-open-cv-using-python/37235130. 

* I have made the following modifications to the code:

    *   Instead of taking a fixed size canvas, I have modifed it to take an RGB image.
    *   Color of the lines being drawn and the final lines being drawn has been changed.
    *   The code now saves roi masked image on the disk.


In [7]:
import numpy as np
import cv2

In [8]:

class PolygonDrawer:
    def __init__(self, window_name, image):
        self.FINAL_LINE_COLOR = (0, 255, 0)
        self.WORKING_LINE_COLOR = (0, 0, 255)
        self.image = image
        self.window_name = window_name
        self.done = False
        self.current = (0, 0)
        self.points = []

    def on_mouse(self, event, x, y, buttons, user_param):
        """
        Mouse callback function that handles mouse events for polygon drawing.
        """
        if self.done:
            return
        if event == cv2.EVENT_MOUSEMOVE:
            self.current = (x, y)
        elif event == cv2.EVENT_LBUTTONDOWN:
            self.add_point(x, y)
        elif event == cv2.EVENT_RBUTTONDOWN:
            self.complete_polygon()

    def add_point(self, x, y):
        """
        Adds a point to the list of polygon points.
        """
        print(f"Adding point #{len(self.points)} with position ({x}, {y})")
        self.points.append((x, y))

    def complete_polygon(self):
        """
        Marks polygon drawing as complete.
        """
        print(f"Completing polygon with {len(self.points)} points.")
        self.done = True

    def draw_polygon(self, canvas):
        """
        Draws the current state of the polygon on the canvas.
        """
        if len(self.points) > 0:
            cv2.polylines(canvas, np.array([self.points]), False, self.FINAL_LINE_COLOR, thickness=4)
            if self.current != (0, 0):
                cv2.line(canvas, self.points[-1], self.current, self.WORKING_LINE_COLOR, thickness=4)
        return canvas

    def run(self):
        """
        Runs the polygon drawing application.
        """
        cv2.namedWindow(self.window_name, flags=cv2.WINDOW_KEEPRATIO)
        cv2.imshow(self.window_name, self.image)
        cv2.waitKey(1)
        cv2.setMouseCallback(self.window_name, self.on_mouse)

        while not self.done:
            canvas = self.image.copy()
            cv2.putText(canvas, "Press 'Esc' to finish adding points", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
            canvas = self.draw_polygon(canvas)
            cv2.imshow(self.window_name, canvas)
            if cv2.waitKey(50) == 27:  # ESC hit
                self.done = True

        mask = np.zeros(self.image.shape[:2], np.uint8)
        if len(self.points) > 0:
            cv2.fillPoly(mask, np.array([self.points]), 255)
            self.image = cv2.bitwise_and(self.image, self.image, mask=mask)

        cv2.imshow(self.window_name, self.image)
        cv2.waitKey()
        cv2.destroyAllWindows()
        return self.image

In [9]:
if __name__ == "__main__":
    print("====> Use left click to draw polygon, right click to release and finish")
    print()
    image = cv2.imread('0.jpg')
    pd = PolygonDrawer(window_name="Draw Polygonal ROI", image=image)
    result_image = pd.run()
    cv2.imwrite("frame_with_polygon.jpg", result_image)
    print("Polygon Points:", pd.points)

====> Use left click to draw polygon, right click to release and finish
Adding point #0 with position (67, 113)
Adding point #1 with position (139, 45)
Adding point #2 with position (308, 38)
Adding point #3 with position (422, 128)
Adding point #4 with position (373, 208)
Adding point #5 with position (232, 231)
Adding point #6 with position (153, 179)
Completing polygon with 7 points.
Polygon Points: [(67, 113), (139, 45), (308, 38), (422, 128), (373, 208), (232, 231), (153, 179)]
